In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

## Initial Data Diagnosis

In [2]:
df = pd.read_csv('creditcard.csv')

In [3]:
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [4]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,1.175161e-15,3.384974e-16,-1.379537e-15,2.094852e-15,1.021879e-15,1.494498e-15,-5.620335e-16,1.149614e-16,-2.414189e-15,...,1.628620e-16,-3.576577e-16,2.618565e-16,4.473914e-15,5.109395e-16,1.686100e-15,-3.661401e-16,-1.227452e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,...,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


- Dataset consists of 31 quantititave variables, with the target being "Class". Apart from "Time" and "Amount", all other variables are quite standardized with mean 0 and standard deviation of roughly 1.

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

- Data is efficiently stored in float64 and int64 values, taking up 67.4 MB

In [6]:
df.isnull().sum()

Time      0
V1        0
V2        0
V3        0
V4        0
V5        0
V6        0
V7        0
V8        0
V9        0
V10       0
V11       0
V12       0
V13       0
V14       0
V15       0
V16       0
V17       0
V18       0
V19       0
V20       0
V21       0
V22       0
V23       0
V24       0
V25       0
V26       0
V27       0
V28       0
Amount    0
Class     0
dtype: int64

In [7]:
print(df.Class.value_counts())
print(df.duplicated().sum())

Class
0    284315
1       492
Name: count, dtype: int64
1081


- No null values but some duplications.

## Data Preparation

### Duplication

In [8]:
df.drop_duplicates(inplace=True)
df.drop("Time", axis=1,inplace=True)
df.shape

(283726, 30)

- Drop identical duplicates for reduced dataset bias and column "Time" to focus on static modeling. 

### Multicollinearity

In [9]:
design_matrix = df.drop("Class", axis=1)
correlation_matrix = design_matrix.corr()

determinant_correlation_matrix = np.linalg.det(correlation_matrix)
print(f"Determinant of the correlation matrix: {determinant_correlation_matrix:.10f}")

Determinant of the correlation matrix: 0.0808995550


- From this determinant value of the correlation matrix, we recognize that the given data has very high multicollinearity. This means that we would need to find the features being highly correlated to other features to ensure that the solver is stable during the fitting process. To do that, we need to utilize VIF.

In [10]:
X = sm.add_constant(design_matrix)

# Create a new DataFrame to store the VIF scores
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns

# Calculate VIF for each variable using a list comprehension
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

print("--- VIF Scores ---")
print(vif_data)

--- VIF Scores ---
   Variable        VIF
0     const   2.529619
1        V1   1.633287
2        V2   4.466661
3        V3   1.542250
4        V4   1.121088
5        V5   2.816732
6        V6   1.572235
7        V7   2.907451
8        V8   1.131088
9        V9   1.024158
10      V10   1.123993
11      V11   1.000245
12      V12   1.001702
13      V13   1.000372
14      V14   1.014246
15      V15   1.000101
16      V16   1.000656
17      V17   1.001508
18      V18   1.015784
19      V19   1.038426
20      V20   2.403376
21      V21   1.135968
22      V22   1.052271
23      V23   1.159593
24      V24   1.000351
25      V25   1.028632
26      V26   1.000168
27      V27   1.011017
28      V28   1.001596
29   Amount  12.295411


- Based on the VIF scores, we would want to eliminate "Amount" first.

In [11]:
design_matrix = df.drop(["Class", "Amount"], axis=1)
correlation_matrix = design_matrix.corr()

determinant_correlation_matrix = np.linalg.det(correlation_matrix)
print(f"Determinant of the correlation matrix: {determinant_correlation_matrix:.10f}")

Determinant of the correlation matrix: 0.9946932395


- We now have a highly non-linearly dependent dataset that will ensure stable solutions.

In [12]:
df.drop("Amount", axis=1,inplace=True)

### Class Imbalance

In [13]:
df['Class'].value_counts(normalize=True) * 100

Class
0    99.83329
1     0.16671
Name: proportion, dtype: float64

- As seen from this proportion, the positive events are extremely dominated by the negative events. To address this, methods like class weights, SMOTE, and undersampling might be implemented.

### Logit Linearity

In [16]:
X = df.drop("Class", axis=1)
Y = df["Class"]

features = X.columns.tolist()

linearity_scores = {}

# --- 2. Loop Through Each Feature and Plot on a Specific Axis ---
for i, predictor_col in enumerate(features):

    # Create a temporary DataFrame for the check
    check_df = pd.DataFrame({
        'predictor': X[predictor_col],
        'target': Y
    })

    # Bin the predictor
    check_df['predictor_bin'] = pd.qcut(check_df['predictor'], q=10, duplicates='drop')

    # Calculate log-odds per bin
    logit_df = check_df.groupby('predictor_bin', observed=False).agg(
        mean_predictor=('predictor', 'mean'),
        n_positive=('target', 'sum'),
        n_total=('target', 'count')
    ).reset_index()

    logit_df['p_positive'] = (logit_df['n_positive'] + 0.5) / (logit_df['n_total'] + 1)
    logit_df['log_odds'] = np.log(logit_df['p_positive'] / (1 - logit_df['p_positive']))

    X_fit = logit_df[['mean_predictor']]
    y_fit = logit_df['log_odds']

    # Fit a linear model to the binned points
    model = LinearRegression()
    model.fit(X_fit, y_fit)
    
    # Get predictions and calculate MSE
    predictions = model.predict(X_fit)
    mse = mean_squared_error(y_fit, predictions)
    
    linearity_scores[predictor_col] = mse

# --- Sort the features by their non-linearity score (higher is worse) ---
sorted_scores = sorted(linearity_scores.items(), key=lambda item: item[1], reverse=True)

print("--- Non-Linearity Scores (Higher MSE = More Non-Linear) ---")
for feature, score in sorted_scores:
    print(f"{feature:<20}: {score:.4f}")

--- Non-Linearity Scores (Higher MSE = More Non-Linear) ---
V17                 : 2.3402
V21                 : 1.3178
V16                 : 1.2128
V27                 : 1.1308
V14                 : 1.1128
V28                 : 1.1039
V7                  : 1.0000
V10                 : 0.9564
V2                  : 0.8646
V8                  : 0.7470
V18                 : 0.6840
V11                 : 0.6641
V19                 : 0.5210
V5                  : 0.4949
V20                 : 0.4706
V6                  : 0.4118
V1                  : 0.4009
V23                 : 0.3537
V9                  : 0.3318
V3                  : 0.2598
V12                 : 0.2449
V26                 : 0.1465
V4                  : 0.1442
V24                 : 0.1169
V25                 : 0.0939
V13                 : 0.0693
V15                 : 0.0520
V22                 : 0.0345


- To improve logistic regression performance further, we can eliminate the top 5 non-linear variables (in the logit sense). 

In [ ]:
df.drop(["V17", "V21", "V16", "V27", "V14"], axis=1, inplace=True)

### Outliers

## Feature Engineering